In [ ]:
#Lib for Preprocessing and load data 
import os
from io import open
import csv
import random
import re
import unicodedata
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import pandas as pd

#Lib for Modeling
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
torch.manual_seed(1)
torch.set_printoptions(precision=30)

In [ ]:
input = [[-0.2, -0.1, 0.1, 1.3],
        [-0.9, 1.4, -0.5, 0.6],
        [-1.3, 1.2, 0.8, 0.7],
        [-0.7, -0.1, -0.1, -2.1],
        [0.6, 0.2, -0.7, 0.5]]

In [ ]:
input = torch.tensor(input)

In [ ]:
input

tensor([[-0.200000002980232238769531250000, -0.100000001490116119384765625000,
          0.100000001490116119384765625000,  1.299999952316284179687500000000],
        [-0.899999976158142089843750000000,  1.399999976158142089843750000000,
         -0.500000000000000000000000000000,  0.600000023841857910156250000000],
        [-1.299999952316284179687500000000,  1.200000047683715820312500000000,
          0.800000011920928955078125000000,  0.699999988079071044921875000000],
        [-0.699999988079071044921875000000, -0.100000001490116119384765625000,
         -0.100000001490116119384765625000, -2.099999904632568359375000000000],
        [ 0.600000023841857910156250000000,  0.200000002980232238769531250000,
         -0.699999988079071044921875000000,  0.500000000000000000000000000000]])

In [ ]:
target = [0]

In [ ]:
target = torch.tensor([target])

In [ ]:
target

tensor([[0]])

In [ ]:
input.shape

torch.Size([5, 4])

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, hidden_size):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(hidden_size, 
                          1,
                          bidirectional=True)
        #self.drop = nn.Dropout(p=0.5)
        for name, param in self.lstm.named_parameters():
          if 'weight' or 'bias' in name:
            nn.init.ones_(param)
        self.fc = nn.Linear(2, 1)
        nn.init.ones_(self.fc.weight)
        nn.init.ones_(self.fc.bias)
        

    def forward(self, input, hidden=None):
        # Convert word indexes to embeddings

        print('\n===EMBEDDED RESULT===\n', input)
        # csv_emb = input.tolist()
        # csv_emb = convertDecimal(csv_emb)

        # Pack padded batch of sequences for RNN module
        #packed = nn.utils.rnn.pack_padded_sequence(input, input_lengths)
        #print('\n===PACK PADDED SEQUENCE===\n', packed)
        
        print("\n=====LSTM's WEIGHTS and BIAS=====\n\n",self.lstm.all_weights)
        print("\n=====Linear's WEIGHTS=====\n\n",self.fc.weight)
        print("\n=====Linear's BIAS=====\n\n",self.fc.bias)
        outputs, hidden = self.lstm(input, hidden)
        
        # print("outputs", outputs)
        # Unpack padding
        #coba = outputs[:,0,4]
        #print("\n=====GRU coba=====\n\n", coba)

        #outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        print("\n=====LSTM OUTPUTS=====\n\n",outputs)
        print("\n=====LSTM HIDDEN=====\n\n",hidden)
        
        # Sum bidirectional GRU outputs
        

        #outputs = outputs[:,:self.hidden_size] + outputs[:,self.hidden_size:]
        #print("\n=====COMBINED GRU OUTPUTS=====\n\n",outputs)


        c = torch.cat((hidden[0], hidden[1]), 0)
        print('new hidden', c)

        print('new hidden', c.shape)
        print(' hidden', c[0].item())

        # h = [[c[0].item(), c[1].item(), c[2].item(), c[3].item()]]
        h = [[c[0].item(), c[1].item()]]


        print('h', h)

        th = torch.tensor(h)

        print('th', th)
        print('th', th.shape)

        th = th.to(torch.float32)



        # output_dropped = self.drop(c)
        # print('new hidden', output_dropped)



        # hidden = self.fc(c)
        hidden = self.fc(th)

        print('Perhitungan Linear : ', hidden)


        hidden = F.sigmoid(hidden)

        print('Klasifikasi Label : ', hidden)

        # text_squeeze = torch.squeeze(hidden, 1)
        # final_output = torch.sigmoid(text_squeeze)


        # hidden = F.softmax(hidden, dim=1)
        # print("bi out", outputs.shape)
        # Return output and final hidden state
        # return final_output
        return hidden
    
    #def init_hidden(self, batch_size):
     #   weight = next(self.parameters()).data
      #  hidden = (weight.new(1, batch_size, self.hidden_size).zero_(),
       #           weight.new(1, batch_size, self.hidden_size).zero_())
       # return hidden

In [ ]:
def train(input, target, net, optimizer, criterion):
    # Zero gradients
    optimizer.zero_grad()

    # Set device options
    input_variable = input.to(device)
    target_variable = target.to(device)
    # input_variable = input
    # target_variable = target

    # Initialize variables
    loss = 0
    print_losses = []

    # Forward pass through encoder
    print('input_variable', input_variable)
    hidden = net(input_variable)

    print('Hidden : ', hidden)
    print('Target : ', target)

    # # Create initial decoder input (start with SOS tokens for each sentence)
    # decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    # decoder_input = decoder_input.to(device)

    # # Set initial decoder hidden state to the encoder's final hidden state
    # #set decoder_hidden if using bidirectional GRU in decoder
    # decoder_hidden = encoder_hidden[:1]

    # #set decoder_hidden if using vanilla GRU in decoder
    # # decoder_hidden = encoder_hidden[:decoder.n_layers]

    # # Determine if we are using teacher forcing this iteration
    # # use_teacher_forcing = (True if torch.rand(2).tolist()[0] < teacher_forcing_ratio 
    # #                       else False)
    # # print('Teacher forcing status:',use_teacher_forcing)

    # # Forward batch of sequences through decoder one time step at a time
    # # if use_teacher_forcing:
    # for t in range(max_target_len):
    #     decoder_output, decoder_hidden = decoder(
    #         decoder_input, decoder_hidden, encoder_outputs
    #     )
    #     # Teacher forcing: next input is current target
    #     decoder_input = target_variable[t].view(1, -1)
    #     # Calculate and accumulate loss
    #     decoder_output = torch.log(decoder_output)
    #     loss = criterion(decoder_output,
    #                                     target_variable[t])
    #     print('targ',target_variable[t])
    #     print('loss',loss)
    #     loss += loss
    #     print_losses.append(loss.item())

    loss_ = criterion(hidden.float(), target.float())
    print('targ',target)
    print('loss',loss_)
    loss += loss_
    print_losses.append(loss.item())

    # else:
    #   for t in range(max_target_len):
    #       decoder_output, decoder_hidden = decoder(
    #           decoder_input, decoder_hidden, encoder_outputs
    #       )
    #       # No teacher forcing: next input is decoder's own current output
    #       _, topi = decoder_output.topk(1)
    #       decoder_input = torch.LongTensor(
    #           [[topi[i][0] for i in range(batch_size)]])
    #       decoder_input = decoder_input.to(device)
    #       # Calculate and accumulate loss
    #       decoder_output = torch.log(decoder_output)
    #       loss = criterion(decoder_output,
    #                                       target_variable[t])
    #       print('targ',target_variable[t])
    #       print('loss',loss)
    #       loss += loss
    #       print_losses.append(loss.item())
# Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    # _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)

    # Adjust model weights
    optimizer.step()

    print('loss :', loss)

    return loss

In [ ]:
hidden_size = 4
batch_size = 1
# clip = 50.0
# teacher_forcing_ratio = 0.1
learning_rate = 0.01
epoch = 1
loss_list = [] 
start_iteration = 0
print_loss = 0

# torch.manual_seed(1)

# Initialize encoder & decoder models
net = BiLSTM(hidden_size)

# Use appropriate device
#encoder = encoder.to(device)

#encoder.train()

# Initialize optimizers
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

criterion = nn.BCELoss()

In [ ]:
pd.set_option('precision', 7)

In [ ]:
for ep in range(epoch):
  #random.shuffle(training_batches)
  # print("Epoch {}".format(ep+1))
  #data train session 
  print('input', input)
  loss = train(input, target, net, optimizer, criterion)
  print('acummulative',loss)
  print_loss += loss
  loss_list=print_loss
  # print("\tIteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration/(len(train_batch)-1) * 100, print_loss))
  print_loss = 0

input tensor([[-0.200000002980232238769531250000, -0.100000001490116119384765625000,
          0.100000001490116119384765625000,  1.299999952316284179687500000000],
        [-0.899999976158142089843750000000,  1.399999976158142089843750000000,
         -0.500000000000000000000000000000,  0.600000023841857910156250000000],
        [-1.299999952316284179687500000000,  1.200000047683715820312500000000,
          0.800000011920928955078125000000,  0.699999988079071044921875000000],
        [-0.699999988079071044921875000000, -0.100000001490116119384765625000,
         -0.100000001490116119384765625000, -2.099999904632568359375000000000],
        [ 0.600000023841857910156250000000,  0.200000002980232238769531250000,
         -0.699999988079071044921875000000,  0.500000000000000000000000000000]])
input_variable tensor([[-0.200000002980232238769531250000, -0.100000001490116119384765625000,
          0.100000001490116119384765625000,  1.299999952316284179687500000000],
        [-0.899999976158